In [1]:
import pandas as pd
import openai
import os
from dotenv import load_dotenv
import requests
from sec_api import ExtractorApi 
from sec_api import XbrlApi

In [2]:

# Load environment variables from .env file
load_dotenv()

# Retrieve the API key from environment variables
api_key = os.environ.get("GPT")

# Check if the API key is retrieved successfully
if api_key:
    print("API loaded")
else:
    print("API key not loaded")
    exit()  # Stop further execution

# Configure OpenAI library to use your API key
openai.api_key = api_key


API loaded


In [6]:
sec_api_key = os.getenv('SEC')
                        

print(sec_api_key)              

156e00bc82b011ad4c4b1c2bf35d7016a8f4e91d73ae2c6d1d06c063caf98a95


In [7]:
import pandas as pd
from sec_api import XbrlApi

# Your API key for the sec-api
API_KEY = sec_api_key
xbrlApi = XbrlApi(API_KEY)


In [30]:
import pandas as pd

def get_balance_sheet(xbrl_json):
    balance_sheet_store = {}

    # Iterate over each US GAAP item in the balance sheet
    for usGaapItem, facts in xbrl_json['BalanceSheets'].items():
        values = []
        indices = []

        # Ensure facts is a list before iterating
        if not isinstance(facts, list):
            print(f"Skipping {usGaapItem} as it's not a list.")
            continue

        for fact in facts:
            # Check if fact is a dictionary
            if not isinstance(fact, dict):
                print(f"Skipping a fact in {usGaapItem} as it's not a dictionary.")
                continue

            # Only consider items without segment. Not required for our analysis.
            if 'segment' not in fact and 'period' in fact and 'instant' in fact['period']:
                # Use 'instant' for index
                index = fact['period']['instant']
                
                # Ensure the 'value' key exists and no index duplicates are created
                if 'value' in fact and index not in indices:
                    values.append(fact['value'])
                    indices.append(index)
                else:
                    print(f"No 'value' key for {usGaapItem} on {index}")

        balance_sheet_store[usGaapItem] = pd.Series(values, index=indices, dtype='float64') 

    balance_sheet = pd.DataFrame(balance_sheet_store)
    # Switch columns and rows so that US GAAP items are rows and each column header represents a date
    return balance_sheet.T 


In [31]:
import pandas as pd

# URLs for Peloton's 10-K filings from 2019 to 2022
url_10k_20 = "https://www.sec.gov/ix?doc=/Archives/edgar/data/1639825/000163982520000122/pton-20200630.htm"
url_10k_21 = "https://www.sec.gov/ix?doc=/Archives/edgar/data/1639825/000163982521000256/pton-20210630.htm"
url_10k_22 = "https://www.sec.gov/ix?doc=/Archives/edgar/data/1639825/000163982522000117/pton-20220630.htm"
url_10k_23 = "https://www.sec.gov/ix?doc=/Archives/edgar/data/1639825/000163982523000132/pton-20230630.htm"

xbrl_json_2020 = xbrlApi.xbrl_to_json(htm_url=url_10k_20)
xbrl_json_2021 = xbrlApi.xbrl_to_json(htm_url=url_10k_21)
xbrl_json_2022 = xbrlApi.xbrl_to_json(htm_url=url_10k_22)
xbrl_json_2023 = xbrlApi.xbrl_to_json(htm_url=url_10k_23)



In [32]:
balance_sheet_2020 = get_balance_sheet(xbrl_json_2020)
balance_sheet_2021 = get_balance_sheet(xbrl_json_2021)
balance_sheet_2022 = get_balance_sheet(xbrl_json_2022)
balance_sheet_2023 = get_balance_sheet(xbrl_json_2023)

Skipping OperatingLeaseRightOfUseAsset as it's not a list.
Skipping DeferredRentCreditNoncurrent as it's not a list.
Skipping BuildToSuitLeaseLiability as it's not a list.
No 'value' key for CommitmentsAndContingencies on 2020-06-30
No 'value' key for CommitmentsAndContingencies on 2019-06-30
No 'value' key for CommitmentsAndContingencies on 2021-06-30
No 'value' key for CommitmentsAndContingencies on 2020-06-30
No 'value' key for CommitmentsAndContingencies on 2022-06-30
No 'value' key for CommitmentsAndContingencies on 2021-06-30
No 'value' key for CommitmentsAndContingencies on 2023-06-30
No 'value' key for CommitmentsAndContingencies on 2022-06-30


In [33]:
print(balance_sheet_2020)

                                                    2017-06-30   2018-06-30  \
CashAndCashEquivalentsAtCarryingValue                      NaN          NaN   
AvailableForSaleSecuritiesDebtSecuritiesCurrent            NaN          NaN   
AccountsReceivableNetCurrent                               NaN          NaN   
InventoryNet                                               NaN          NaN   
PrepaidExpenseAndOtherAssetsCurrent                        NaN          NaN   
AssetsCurrent                                              NaN          NaN   
PropertyPlantAndEquipmentNet                               NaN          NaN   
FiniteLivedIntangibleAssetsNet                             NaN          NaN   
Goodwill                                                   NaN    4200000.0   
RestrictedCashNoncurrent                                   NaN          NaN   
OtherAssetsNoncurrent                                      NaN          NaN   
Assets                                              

In [34]:
balance_sheets_merged = pd.concat([balance_sheet_2020, balance_sheet_2021, balance_sheet_2022, balance_sheet_2023], axis=0, sort=False)

balance_sheets_merged = balance_sheets_merged.sort_index().reset_index()

balance_sheets_merged = balance_sheets_merged.applymap(lambda x: pd.to_numeric(x, errors= 'ignore'))

balance_sheets_merged.head(10)

,index,2017-06-30,2018-06-30,2019-06-30,2020-06-30,2021-06-30,2022-06-30,2023-06-30
0,AccountsPayableAndAccruedLiabilitiesCurrent,NaN,NaN,NaN,361700000.0,989100000.0,NaN,NaN
1,AccountsPayableAndAccruedLiabilitiesCurrent,NaN,NaN,NaN,NaN,989100000.0,797400000.0,NaN
2,AccountsPayableAndAccruedLiabilitiesCurrent,NaN,NaN,NaN,NaN,NaN,797400000.0,478400000.0
3,AccountsPayableCurrent,NaN,NaN,92200000.0,135800000.0,NaN,NaN,NaN
4,AccountsReceivableNetCurrent,NaN,NaN,18500000.0,34600000.0,NaN,NaN,NaN
5,AccountsReceivableNetCurrent,NaN,NaN,NaN,NaN,71400000.0,83600000.0,NaN
6,AccountsReceivableNetCurrent,NaN,NaN,NaN,34600000.0,71400000.0,NaN,NaN
7,AccountsReceivableNetCurrent,NaN,NaN,NaN,NaN,NaN,83600000.0,97200000.0
8,AccruedLiabilitiesCurrent,NaN,NaN,104500000.0,225900000.0,NaN,NaN,NaN
9,AccumulatedOtherComprehensiveIncomeLossNetOfTax,NaN,NaN,NaN,10100000.0,18200000.0,NaN,NaN


In [37]:


# Aggregate by index and take max
balance_sheets = balance_sheets_merged.groupby('index').max()

# Reindex
balance_sheets = balance_sheets.reindex(balance_sheet_2020.index)

# Drop columns before 2019 and filter out non-annual data
cols_to_drop = []
for col in balance_sheets.columns:
    splitted = col.split('-')
    start = '-'.join(splitted[:3])
    end = '-'.join(splitted[3:])
    start_date = pd.to_datetime(start)
    end_date = pd.to_datetime(end)
    duration = (end_date - start_date).days / 360

    # Drop if duration is less than a year or if the year is before 2019
    if duration < 1 or start_date.year < 2019:
        cols_to_drop.append(col)

balance_sheets.drop(columns=cols_to_drop, inplace=True)

# Convert to readable format
balance_sheets = balance_sheets.apply(lambda row: pd.to_numeric(row, errors='coerce', downcast='integer').astype(str), axis=1)

# Sort columns
balance_sheets = balance_sheets[sorted(balance_sheets.columns)]

print("Cleaned Balance sheets from 10-K filings (2019 onwards) as dataframe:")
print('----------------------------------------------------------------------')
balance_sheets


Cleaned Balance sheets from 10-K filings (2019 onwards) as dataframe:
----------------------------------------------------------------------


,2019-06-30,2020-06-30,2021-06-30,2022-06-30,2023-06-30
CashAndCashEquivalentsAtCarryingValue,162100000,1035500000,1134800000,1253900000,813900000
AvailableForSaleSecuritiesDebtSecuritiesCurrent,216000000.0,719500000.0,472000000.0,0.0,nan
AccountsReceivableNetCurrent,18500000,34600000,71400000,83600000,97200000
InventoryNet,136600000,244500000,937100000,1104500000,522600000
PrepaidExpenseAndOtherAssetsCurrent,48400000,124500000,202800000,192500000,205400000
AssetsCurrent,581700000,2158600000,2818100000,2634600000,1639100000
PropertyPlantAndEquipmentNet,249700000,242300000,591900000,610900000,444800000
FiniteLivedIntangibleAssetsNet,19500000,16000000,247900000,41300000,25600000
Goodwill,4300000,39100000,210100000,41200000,41200000
RestrictedCashNoncurrent,800000,1500000,900000,3800000,71600000


In [40]:
def analyze_balance_sheet_with_gpt(df):
    # Convert DataFrame to a string representation for sending to GPT
    balance_sheet_str = df.to_string()

    prompt_text = f"Please analyze the following balance sheet data for the last few years:\n\n{balance_sheet_str}\n\nProvide insights on the assets, liabilities, and equity trends, and evaluate if the investing risk has increased in 750 words or less."

    # Make API call to OpenAI
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=prompt_text,
        max_tokens=1000
    )

    # Print GPT's analysis
    print(response.choices[0].text.strip())

In [41]:
analyze_balance_sheet_with_gpt(balance_sheets)

Overall, the balance sheet data for the last few years indicates a mixed picture for the company. Assets have generally declined over the past few years, going from 86.45 billion in 2019 to 27.69 billion in 2023. Liabilities have trended downwards as well, with a drop from a high of 130.38 billion in 2020 to 30.64 billion in 2023. Meanwhile, equity has fluctuated, reaching an all-time low of -295.1 million in 2023.

Cash and cash equivalents have experienced the biggest decline, falling from 162.1 billion in 2019 to 81.39 billion in 2023. This could indicate that the company has experienced cash flow issues over the last few years, likely due to the pandemic and resulting economic recession. Other current assets, such as Accounts Receivable, Inventory, and Prepaid Expenses, have likewise declined over the same period.

This trend is partially offset by an increase in Noncurrent Assets, specifically Property, Plant, and Equipment (PPE) and Finite-Lived Intangible Assets (FLIAs). PPE inc